1. Import Libraries:
- The necessary libraries such as time, numpy, pandas, os, spacy, sklearn, ast, and others are imported.

In [2]:
import time
import numpy as np
import pandas as pd
import os

import spacy
import en_core_web_sm

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

from sklearn.model_selection import train_test_split
import ast

import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans

from spacy.training import Example
import random
from tqdm import tqdm

C:\Users\SunagatullinAyaz\.conda\envs\pythonProject2\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
# Print file paths in the 'dataset' directory
for dirname, _, filenames in os.walk('dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

dataset\NER_dataset.csv


In [4]:
ner_dataset = pd.read_csv("dataset/NER_dataset.csv", encoding="latin1")
ner_dataset

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [5]:
ner_dataset.describe()

,Sentence #,Word,POS,Tag
count,47959,1048575,1048575,1048575
unique,47959,35178,42,17
top,Sentence: 1,the,NN,O
freq,1,52573,145807,887908


In [6]:
ner_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Sentence #  47959 non-null    object
 1   Word        1048575 non-null  object
 2   POS         1048575 non-null  object
 3   Tag         1048575 non-null  object
dtypes: object(4)
memory usage: 32.0+ MB


In [7]:
ner_dataset["POS"].value_counts()

NN      145807
NNP     131426
IN      120996
DT       98454
JJ       78412
NNS      75840
.        47831
VBD      39379
,        32757
VBN      32328
VBZ      24960
CD       24695
VB       24211
CC       23716
TO       23061
RB       20252
VBG      19125
VBP      16158
PRP      13318
POS      11257
PRP$      8655
MD        6973
``        3728
WDT       3698
JJS       3034
JJR       2967
WP        2542
NNPS      2521
RP        2490
WRB       2184
$         1149
RBR       1055
:          795
RRB        679
LRB        678
EX         663
RBS        296
;          214
PDT        147
WP$         99
UH          24
FW           1
Name: POS, dtype: int64

In [8]:
ner_dataset["Tag"].value_counts()

O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: Tag, dtype: int64

In [3]:
dfRaw = pd.read_csv(
    "https://raw.githubusercontent.com/orico/SpacyVsStanfordNER/master/data/us_cities_states_counties.csv")
dfRaw.head()

,City,State short,State full,County,City alias
0,Holtsville,NY,New York,SUFFOLK,Internal Revenue Service
1,Holtsville,NY,New York,SUFFOLK,Holtsville
2,Adjuntas,PR,Puerto Rico,ADJUNTAS,URB San Joaquin
3,Adjuntas,PR,Puerto Rico,ADJUNTAS,Jard De Adjuntas
4,Adjuntas,PR,Puerto Rico,ADJUNTAS,Colinas Del Gigante


In [4]:
print("The shape of dataframe is: ", dfRaw.shape)
print("Unique number of StateShort are:", len(dfRaw.City.unique()))
print("Unique number of County are:", len(dfRaw['County'].unique()))
print("Unique number of City Alias are:", len(dfRaw['City alias'].unique()))

The shape of dataframe is:  (63211, 5)
Unique number of StateShort are: 18857
Unique number of County are: 1932
Unique number of City Alias are: 39891


In [5]:
# Drop rows with missing values
dfRaw.dropna(inplace=True)
dfRaw.isna().any()

City           False
State short    False
State full     False
County         False
City alias     False
dtype: bool

In [6]:
City_modified = dfRaw.City.sample(500)

Print Results:
- The script prints the time taken for spaCy processing, the number of unique cities in the original dataset,
the number of unique cities spaCy obtained, and the percentage of data recognized by spaCy.


In [7]:
count = set()
stime = time.time()

for i, city in tqdm(enumerate(dfRaw['City'].values)):
    doc = nlp(city)
    for x in doc.ents:
        # Extract 'GPE' (geopolitical entity) entities and add to the set
        if x.label_ == 'GPE':
            count.add(x)
print("Time taken for spaCy: ", time.time() - stime, "seconds")
print("Number of Unique City we had: ", len(dfRaw['City'].unique()))
print("Number of Unique City spaCy obtained: ", len(count))
print("Percentage of data recognised", len(count) * 100 / len(dfRaw['City'].unique()), '%')

63203it [05:04, 207.70it/s]

Time taken for spaCy:  304.31031942367554 seconds
Number of Unique City we had:  18852
Number of Unique City spaCy obtained:  15132
Percentage of data recognised 80.26734563971992 %
